In [1]:
# Dinh Hoang Viet Phuong - 301123263

In [2]:
import pandas as pd

# Path to the data
path = "/home/centos/data/retail-data/by-day/"

# Load 2 days of data
df_09 = pd.read_csv(path + "2011-01-09.csv", infer_datetime_format=True)
df_10 = pd.read_csv(path + "2011-01-10.csv", infer_datetime_format=True)

# Combine the two dataframes
df_phuong = pd.concat([df_09, df_10], ignore_index=True)

print(df_phuong.head())

  InvoiceNo StockCode                       Description  Quantity  \
0    540498     21485  RETROSPOT HEART HOT WATER BOTTLE         1   
1    540498     22112        CHOCOLATE HOT WATER BOTTLE         1   
2    540498    17091A           LAVENDER INCENSE IN TIN         6   
3    540498     84813     SET OF 4 DIAMOND NAPKIN RINGS         6   
4    540498     22795      SWEETHEART RECIPE BOOK STAND         3   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2011-01-09 10:06:00       4.95     17243.0  United Kingdom  
1  2011-01-09 10:06:00       4.95     17243.0  United Kingdom  
2  2011-01-09 10:06:00       1.25     17243.0  United Kingdom  
3  2011-01-09 10:06:00      12.75     17243.0  United Kingdom  
4  2011-01-09 10:06:00       6.75     17243.0  United Kingdom  


In [4]:
from pyspark.sql import SparkSession
import time

spark = SparkSession.builder.appName("retailDataAnalysis").getOrCreate()

start_time = time.time()

df_phuong = spark.read.option("inferSchema", "true").csv(["/home/centos/data/retail-data/by-day/2011-01-09.csv", 
                                                          "/home/centos/data/retail-data/by-day/2011-01-10.csv"])

end_time = time.time()

loading_time = end_time - start_time
print(f"Time taken to load the data: {loading_time} seconds")

df_phuong.show(truncate=False)

Time taken to load the data: 2.8408827781677246 seconds
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|_c0      |_c1      |_c2                                |_c3     |_c4                |_c5      |_c6       |_c7           |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
|540549   |21587    |null                               |-53     |2011-01-10 09:07:00|0.0      |null      |United Kingdom|
|540550   |22962    |JAM JAR WITH PINK LID              |12      |2011-01-10 09:11:00|0.85     |12539.0   |Spain         |
|540550   |22961    |JAM MAKING SET PRINTED             |12      |2011-01-10 09:11:00|1.45     |12539.0   |Spain         |
|540550   |22960    |JAM MAKING SET WITH JARS           |6       |2011-01-10 09:11:

In [5]:
from pyspark.sql import functions as F

# Filter the dataframe
filtered_df = df_phuong.filter(
    (df_phuong["_c1"].startswith("227")) &
    ((df_phuong["_c2"].contains("ALARM CLOCK")) | (df_phuong["_c5"] > 5))
)

# Show the filtered transactions
filtered_df.show()

+------+-----+--------------------+---+-------------------+-----+-------+--------------+
|   _c0|  _c1|                 _c2|_c3|                _c4|  _c5|    _c6|           _c7|
+------+-----+--------------------+---+-------------------+-----+-------+--------------+
|540550|22785|SQUARECUSHION COV...|  2|2011-01-10 09:11:00| 6.75|12539.0|         Spain|
|540551|22720|SET OF 3 CAKE TIN...|  1|2011-01-10 09:43:00|11.02|   null|United Kingdom|
|540551|22722|SET OF 6 SPICE TI...|  2|2011-01-10 09:43:00| 8.47|   null|United Kingdom|
|540551|22723|SET OF 6 HERB TIN...|  1|2011-01-10 09:43:00| 8.47|   null|United Kingdom|
|540551|22749|FELTCRAFT PRINCES...|  1|2011-01-10 09:43:00| 7.62|   null|United Kingdom|
|540551|22767|TRIPLE PHOTO FRAM...|  1|2011-01-10 09:43:00|20.38|   null|United Kingdom|
|540551|22795|SWEETHEART RECIPE...|  1|2011-01-10 09:43:00|13.57|   null|United Kingdom|
|540556|22781|GUMBALL MAGAZINE ...|  4|2011-01-10 09:58:00| 7.65|   null|        Israel|
|540640|22762|CUPBOAR

In [6]:
# Filter the dataframe and store the results in df2_phuong
df2_phuong = df_phuong.filter(
    (df_phuong["_c1"].startswith("227")) &
    ((df_phuong["_c2"].contains("ALARM CLOCK")) | (df_phuong["_c5"] > 5))
)

df2_phuong.show()

+------+-----+--------------------+---+-------------------+-----+-------+--------------+
|   _c0|  _c1|                 _c2|_c3|                _c4|  _c5|    _c6|           _c7|
+------+-----+--------------------+---+-------------------+-----+-------+--------------+
|540550|22785|SQUARECUSHION COV...|  2|2011-01-10 09:11:00| 6.75|12539.0|         Spain|
|540551|22720|SET OF 3 CAKE TIN...|  1|2011-01-10 09:43:00|11.02|   null|United Kingdom|
|540551|22722|SET OF 6 SPICE TI...|  2|2011-01-10 09:43:00| 8.47|   null|United Kingdom|
|540551|22723|SET OF 6 HERB TIN...|  1|2011-01-10 09:43:00| 8.47|   null|United Kingdom|
|540551|22749|FELTCRAFT PRINCES...|  1|2011-01-10 09:43:00| 7.62|   null|United Kingdom|
|540551|22767|TRIPLE PHOTO FRAM...|  1|2011-01-10 09:43:00|20.38|   null|United Kingdom|
|540551|22795|SWEETHEART RECIPE...|  1|2011-01-10 09:43:00|13.57|   null|United Kingdom|
|540556|22781|GUMBALL MAGAZINE ...|  4|2011-01-10 09:58:00| 7.65|   null|        Israel|
|540640|22762|CUPBOAR

In [7]:
from pyspark.sql import functions as F

summary_stats = df2_phuong.agg(
    F.sum("_c3").alias("Total_Quantity"),
    F.min("_c3").alias("Minimum_Quantity"),
    F.max("_c3").alias("Maximum_Quantity")
)

summary_stats.show()

+--------------+----------------+----------------+
|Total_Quantity|Minimum_Quantity|Maximum_Quantity|
+--------------+----------------+----------------+
|         184.0|               1|               8|
+--------------+----------------+----------------+



In [8]:
outside_uk = df2_phuong.filter(F.col("_c7") != "United Kingdom")

outside_uk.show()

+------+-----+--------------------+---+-------------------+----+-------+------+
|   _c0|  _c1|                 _c2|_c3|                _c4| _c5|    _c6|   _c7|
+------+-----+--------------------+---+-------------------+----+-------+------+
|540550|22785|SQUARECUSHION COV...|  2|2011-01-10 09:11:00|6.75|12539.0| Spain|
|540556|22781|GUMBALL MAGAZINE ...|  4|2011-01-10 09:58:00|7.65|   null|Israel|
|540642|22725|ALARM CLOCK BAKEL...|  8|2011-01-10 13:22:00|3.75|12681.0|France|
|540642|22726|ALARM CLOCK BAKEL...|  8|2011-01-10 13:22:00|3.75|12681.0|France|
|540642|22727|ALARM CLOCK BAKEL...|  8|2011-01-10 13:22:00|3.75|12681.0|France|
|540642|22728|ALARM CLOCK BAKEL...|  8|2011-01-10 13:22:00|3.75|12681.0|France|
|540642|22729|ALARM CLOCK BAKEL...|  8|2011-01-10 13:22:00|3.75|12681.0|France|
|540642|22730|ALARM CLOCK BAKEL...|  8|2011-01-10 13:22:00|3.75|12681.0|France|
|540642|22781|GUMBALL MAGAZINE ...|  6|2011-01-10 13:22:00|7.65|12681.0|France|
+------+-----+--------------------+---+-